In [1]:
import tensorflow as tf

tf.keras.backend.clear_session()  # For easy reset of notebook state.

**Part 1: Saving Sequential models or Functional models**

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='3_layer_mlp')
model.summary()

Model: "3_layer_mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=keras.optimizers.RMSprop())
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1)

# Reset metrics before saving so that loaded model has same state,
# since metric states are not preserved by Model.save_weights
model.reset_metrics()

Train on 60000 samples
60000/60000 [==============================] - 2s 31us/sample - loss: 0.3158


In [4]:
# Save predictions for future checks
predictions = model.predict(x_test)

**Whole-model saving**

可以使用 Functional API 将模型保存成单独的文件，之后可以从该文件重建模型，而不需要创建模型的代码。文件包括：
* 模型结构
* 模型权重（训练过程中学习得到）
* 训练配置（compile 中的内容）
* 优化器和其状态（可以重新训练）

In [5]:
# Save the model
model.save('path_to_my_model.h5')

# # Recreate the exact same model purely from the file
new_model = keras.models.load_model('path_to_my_model.h5')

In [6]:
import numpy as np

# Check that the state is preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Note that the optimizer state is preserved as well:
# you can resume training where you left off.

**Export to SavedModel**

也可以将整个模型导出为 Tensorflow SavedModel 格式，SavedModel 是 Tensorflow serving 支持的。

In [7]:
# Export the model to a SavedModel
model.save('path_to_saved_model', save_format='tf')

# Recreate the exact same model
new_model = keras.models.load_model('path_to_saved_model')

# Check that the state is preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: path_to_saved_model\assets


**只保存结构**

有时候，只想保存模型的结构，不想保存模型权重或优化器，这时候可以使用使用 get_config() 来获取模型的 "config"。config 是 Python dict，可以重建模型--重新初始化，没有之前训练学到的信息。

In [8]:
config = model.get_config()
config

{'name': '3_layer_mlp',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 784),
    'dtype': 'float32',
    'sparse': False,
    'name': 'digits'},
   'name': 'digits',
   'inbound_nodes': []},
  {'class_name': 'Dense',
   'config': {'name': 'dense_1',
    'trainable': True,
    'dtype': 'float32',
    'units': 64,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'name': 'dense_1',
   'inbound_nodes': [[['digits', 0, 0, {}]]]},
  {'class_name': 'Dense',
   'config': {'name': 'dense_2',
    'trainable': True,
    'dtype': 'float32',
    'units': 64,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUnif

In [9]:
reinitialized_model = keras.Model.from_config(config)

# Note that the model state is not preserved! We only saved the architecture.
new_predictions = reinitialized_model.predict(x_test)
assert abs(np.sum(predictions - new_predictions)) > 0.

也可以使用 to_json() from_json()，这种方法将模型保存成 json 串的形式。适合于将模型结构保存到文件中。

In [10]:
json_config = model.to_json()
print(json_config)
reinitialized_model = keras.models.model_from_json(json_config)

{"class_name": "Model", "config": {"name": "3_layer_mlp", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 784], "dtype": "float32", "sparse": false, "name": "digits"}, "name": "digits", "inbound_nodes": []}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "name": "dense_1", "inbound_nodes": [[["digits", 0, 0, {}]]]}, {"class_name": "Dense", "config": {"name": "dense_2", "trainable": true, "dtype": "float32", "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "con

**只保存权重**

如果只想保存权重，不保存模型结构，可以使用 get_weights() 和 set_weights()：

In [11]:
weights = model.get_weights()  # Retrieves the state of the model
model.set_weights(weights)  # Sets the state of the model

可以结合使用 get_config()/from_config() 和 get_weights()/set_weights() 来重建模型。但是，不同于 model.save()，这种方法不包括训练配置和优化器，如果要接着训练必须先调用 compile()。

In [12]:
config = model.get_config()
weights = model.get_weights()

new_model = keras.Model.from_config(config)
new_model.set_weights(weights)

# Check that the state is preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

get_weights() 和 set_weights() 同样有保存成文件的版本：save_weights(fpath) 和 load_weights(fpath)

In [13]:
# Save JSON config to disk
json_config = model.to_json()
with open('model_config.json', 'w') as json_file:
    json_file.write(json_config)
# Save weights to disk
model.save_weights('path_to_my_weights.h5')

# Reload the model from the 2 files we saved
with open('model_config.json') as json_file:
    json_config = json_file.read()
new_model = keras.models.model_from_json(json_config)
new_model.load_weights('path_to_my_weights.h5')

# Check that the state is preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Note that the optimizer was not preserved.

记住，最简单，也是推荐的方式还是使用 save():

In [14]:
model.save('path_to_my_model.h5')
del model
model = keras.models.load_model('path_to_my_model.h5')

**使用 Tensorflow checkpoints 只保存权重**

save_weights 可以保存成 HDF5 格式，也可以保存成 tf checkpoint 格式，取决于文件的后缀，如果后缀为 ".h5" 或 ".keras"，保存成 HDF5 格式，否则为 Checkpoint，也可以通过 save_format 参数指定格式，参数值为 "tf" 或 "h5"。

In [15]:
model.save_weights('path_to_my_tf_checkpoint')  # 等价于下面
model.save_weights('path_to_my_tf_checkpoint', save_format='tf')

**Part 2: Saving and Loading of Subclassed Models**

Sequential 模型和 Functional 模型是 datastructures that represent a DAG of layers，所以它们可以安全地序列化和反序列化。A subclassed 模型不同之处是它不同 datastructure，只是一段代码。模型的结构在 call 方法中定义，这意味着模型结构不能被安全序列化。导入模型时，必须访问创建模型的代码。

下面的 subclassed 模型，结构和上面的一致：

In [16]:
class ThreeLayerMLP(keras.Model):
    def __init__(self, name=None):
        super(ThreeLayerMLP, self).__init__(name=name)
        self.dense_1 = layers.Dense(64, activation='relu', name='dense_1')
        self.dense_2 = layers.Dense(64, activation='relu', name='dense_2')
        self.pred_layer = layers.Dense(10, name='predictions')
        
    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        return self.pred_layer(x)
    
def get_model():
    return ThreeLayerMLP(name='3_layer_mlp')

model = get_model()

需要记住一点：没有被用过的 subclassed model 无法被保存，这是因为 subclassed model 需要在一些数据上调用来创建权重。直到被调用，模型才会知道输入数据的 shape 和 dtype。而在 Functional 模型中，输入的 shape 和 dtype 是提前指定的（通过 keras.Input(...)），所以 Functional 模型被初始化后就可以被保存。

In [18]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=keras.optimizers.RMSprop())
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1)
# Reset metrics before saving so that loaded model has same state,
# since metric states are not preserved by Model.save_weights
model.reset_metrics()

Train on 60000 samples
60000/60000 [==============================] - 2s 28us/sample - loss: 0.3026


有三种方法可以保存、恢复 subclassed model：

**Approach 1:**

保存 subclassed model 的推荐方式是使用 save_weights 来创建 TensorFlow SavedModel checkpoint，会包含模型所有变量的值：
* The layers' weights
* The optimizer's state
* Any variables associated with stateful model metrics (if any)

In [19]:
model.save_weights('path_to_my_weights', save_format='tf')

In [20]:
# Save predictions for future checks
predictions = model.predict(x_test)
# Also save the loss on the first batch
# to later assert that the optimizer state was preserved
first_batch_loss = model.train_on_batch(x_train[:64], y_train[:64])

为了恢复模型，需要访问创建模型的代码，为了恢复 optimizer 状态和 the state of any stateful metric，需要在调用 load_weights 之前编译模型（配置必须一致），并用一些数据来调用它（如下面的 train_on_batch）。

In [21]:
# Recreate the model
new_model = get_model()
new_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=keras.optimizers.RMSprop())

# This initializes the variables used by the optimizers,
# as well as any stateful metric variables
new_model.train_on_batch(x_train[:1], y_train[:1])

# Load the state of the old model
new_model.load_weights('path_to_my_weights')

# Check that the model state has been preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# The optimizer state is preserved as well,
# so you can resume training where you left off
new_first_batch_loss=new_model.train_on_batch(x_train[:64], y_train[:64])
assert first_batch_loss == new_first_batch_loss

**Approach 2:**

第二种方法是使用 model.save 保存整个模型，然后使用 load_model 来恢复模型：

In [24]:
# Save the model
model.save('path_to_my_model', save_format='tf')

# Recreate the exact same model purely from the file
new_model = keras.models.load_model('path_to_my_model')

INFO:tensorflow:Assets written to: path_to_my_model\assets


**Approach 3:**

第三种方法是使用 tf.saved_model.save，这种方法等价于 model.save 的 tf 格式。然后调用 load_model 来恢复：

In [25]:
# Save the model
tf.saved_model.save(model, 'my_saved_model')
# Restoring the model
restored_saved_model = keras.models.load_model('my_saved_model')

INFO:tensorflow:Assets written to: my_saved_model\assets
